# OData Queries with Digital Thread Services

OData is a powerful tool that can be used with the NI Measurement Data Store to access stored measurement data and metadata.

This notebook demonstrates how to use OData queries with the NI Measurement Data Store. It covers basic queries and some more advanced query capabilities including filtering, expansion, ordering, and aggregation.

## Prerequisites

- NI Measurement Data Store running and accessible
- Python environment with the `ni.datastore` package
- Sample test data in the datastore


## Sample Data Setup

Before demonstrating OData queries, let's create some structured test data. This section sets up TestResults, Steps, and Measurements that we'll query in the following sections.

This mimics a typical test scenario where we test multiple electronic components with various measurements at different test steps.

In [40]:
from ni.datastore.metadata import (
    MetadataStoreClient,
    HardwareItem,
    Operator,
    SoftwareItem,
    TestStation,
    Uut,
    UutInstance,
)
from ni.datastore.data import DataStoreClient, TestResult, Step
from nitypes.scalar import Scalar
from nitypes.waveform import AnalogWaveform, Timing
from datetime import timezone
import hightime as ht
import numpy as np

# Create metadata store client
metadata_store_client = MetadataStoreClient()

# Create aliases for reusable metadata
ALIAS_OPERATOR_SMITH = "Operator_Smith"
ALIAS_TEST_STATION_A1 = "TestStation_A1"
ALIAS_UUT_POWER_SUPPLY = "UUT_PowerSupply_v2"
ALIAS_UUT_AMPLIFIER = "UUT_Amplifier_v1"
ALIAS_DMM = "DMM_PXIe4081"
ALIAS_SCOPE = "Scope_PXIe5171"
ALIAS_PYTHON_ENV = "Python_3.11"

print("Setting up metadata...")

# Create operator
operator = Operator(operator_name="Alex Smith", role="Test Engineer")
metadata_store_client.create_operator(operator)
metadata_store_client.create_alias(ALIAS_OPERATOR_SMITH, operator)

# Create test station
test_station = TestStation(test_station_name="TestStation_A1")
metadata_store_client.create_test_station(test_station)
metadata_store_client.create_alias(ALIAS_TEST_STATION_A1, test_station)

# Create UUTs (products being tested)
power_supply_uut = Uut(model_name="PowerSupply v2.1", family="Power")
metadata_store_client.create_uut(power_supply_uut)
metadata_store_client.create_alias(ALIAS_UUT_POWER_SUPPLY, power_supply_uut)

amplifier_uut = Uut(model_name="Audio Amplifier v1.3", family="Audio")
metadata_store_client.create_uut(amplifier_uut)
metadata_store_client.create_alias(ALIAS_UUT_AMPLIFIER, amplifier_uut)

# Create test equipment
dmm = HardwareItem(manufacturer="NI", model="PXIe-4081", serial_number="DMM001")
metadata_store_client.create_hardware_item(dmm)
metadata_store_client.create_alias(ALIAS_DMM, dmm)

scope = HardwareItem(manufacturer="NI", model="PXIe-5171", serial_number="SCOPE001")
metadata_store_client.create_hardware_item(scope)
metadata_store_client.create_alias(ALIAS_SCOPE, scope)

# Create software item
python_env = SoftwareItem(product="Python", version="3.11.5")
metadata_store_client.create_software_item(python_env)
metadata_store_client.create_alias(ALIAS_PYTHON_ENV, python_env)

print("Metadata setup complete!")

Setting up metadata...
Metadata setup complete!


In [27]:
# Create test instances and test results
data_store_client = DataStoreClient()

print("Creating test instances and results...")

# Create UUT instances for testing
power_supply_instance = UutInstance(
    uut_id=ALIAS_UUT_POWER_SUPPLY, 
    serial_number="PS-2024-001"
)
ps_instance_id = metadata_store_client.create_uut_instance(power_supply_instance)

amplifier_instance = UutInstance(
    uut_id=ALIAS_UUT_AMPLIFIER, 
    serial_number="AMP-2024-042"
)
amp_instance_id = metadata_store_client.create_uut_instance(amplifier_instance)

# Create TestResult for Power Supply testing
power_supply_test = TestResult(
    uut_instance_id=ps_instance_id,
    operator_id=ALIAS_OPERATOR_SMITH,
    test_station_id=ALIAS_TEST_STATION_A1,
    software_item_ids=[ALIAS_PYTHON_ENV],
    hardware_item_ids=[ALIAS_DMM, ALIAS_SCOPE],
    test_result_name="Power Supply Validation Test"
)
ps_test_result_id = data_store_client.create_test_result(power_supply_test)

# Create TestResult for Amplifier testing
amplifier_test = TestResult(
    uut_instance_id=amp_instance_id,
    operator_id=ALIAS_OPERATOR_SMITH,
    test_station_id=ALIAS_TEST_STATION_A1,
    software_item_ids=[ALIAS_PYTHON_ENV],
    hardware_item_ids=[ALIAS_DMM, ALIAS_SCOPE],
    test_result_name="Audio Amplifier Performance Test"
)
amp_test_result_id = data_store_client.create_test_result(amplifier_test)

print(f"Created TestResults: PS={ps_test_result_id}, AMP={amp_test_result_id}")

Creating test instances and results...
Created TestResults: PS=2a9b0fe8-d91a-4d76-86ef-dac42c820407, AMP=f679b40d-acd1-4600-8d55-6e2147085add


In [28]:
# Create Steps for Power Supply Test
print("Creating test steps and measurements...")

# Power Supply Test Steps
ps_power_on_step = Step(step_name="Power On Sequence", test_result_id=ps_test_result_id)
ps_power_on_step_id = data_store_client.create_step(ps_power_on_step)

ps_voltage_test_step = Step(step_name="Output Voltage Verification", test_result_id=ps_test_result_id)  
ps_voltage_step_id = data_store_client.create_step(ps_voltage_test_step)

ps_current_test_step = Step(step_name="Current Limit Test", test_result_id=ps_test_result_id)
ps_current_step_id = data_store_client.create_step(ps_current_test_step)

ps_ripple_step = Step(step_name="Output Ripple Measurement", test_result_id=ps_test_result_id)
ps_ripple_step_id = data_store_client.create_step(ps_ripple_step)

# Amplifier Test Steps  
amp_gain_step = Step(step_name="Gain Measurement", test_result_id=amp_test_result_id)
amp_gain_step_id = data_store_client.create_step(amp_gain_step)

amp_frequency_step = Step(step_name="Frequency Response", test_result_id=amp_test_result_id)
amp_freq_step_id = data_store_client.create_step(amp_frequency_step)

amp_distortion_step = Step(step_name="THD Analysis", test_result_id=amp_test_result_id)
amp_thd_step_id = data_store_client.create_step(amp_distortion_step)

print("Created all test steps successfully!")

Creating test steps and measurements...
Created all test steps successfully!


In [29]:
# Create sample measurements with different data types

# Helper function to create waveform data
def create_sample_waveform(amplitude, frequency, duration_ms=10, sample_rate=1000):
    """Create a sample sine wave for testing"""
    samples = int(duration_ms * sample_rate / 1000)
    time_points = np.linspace(0, duration_ms/1000, samples)
    data = amplitude * np.sin(2 * np.pi * frequency * time_points)
    
    return AnalogWaveform(
        sample_count=len(data),
        raw_data=data,
        timing=Timing.create_with_regular_interval(
            ht.timedelta(seconds=1.0/sample_rate),
            ht.datetime.now(timezone.utc)
        )
    )

# Power Supply Measurements
print("Publishing Power Supply measurements...")

# Scalar measurements (DC values)
data_store_client.publish_measurement("Output_Voltage_5V", Scalar(5.02, "V"), ps_voltage_step_id)
data_store_client.publish_measurement("Output_Voltage_12V", Scalar(12.01, "V"), ps_voltage_step_id)
data_store_client.publish_measurement("Current_Limit", Scalar(2.98, "A"), ps_current_step_id)
data_store_client.publish_measurement("Efficiency", Scalar(87.5, "%"), ps_current_step_id)

# Waveform measurements (AC ripple)
ripple_waveform = create_sample_waveform(0.05, 120, duration_ms=50)  # 120Hz ripple
data_store_client.publish_measurement("Ripple_Waveform_5V", ripple_waveform, ps_ripple_step_id)

ripple_waveform_12v = create_sample_waveform(0.08, 120, duration_ms=50)
data_store_client.publish_measurement("Ripple_Waveform_12V", ripple_waveform_12v, ps_ripple_step_id)

# Amplifier Measurements  
print("Publishing Amplifier measurements...")

# Gain at different frequencies
data_store_client.publish_measurement("Gain_1kHz", Scalar(39.8, "dB"), amp_gain_step_id)
data_store_client.publish_measurement("Gain_10kHz", Scalar(39.2, "dB"), amp_gain_step_id)

# Frequency response waveform
freq_response = create_sample_waveform(1.0, 1000, duration_ms=100)  # Frequency sweep response
data_store_client.publish_measurement("Frequency_Response", freq_response, amp_freq_step_id)

# THD measurements
data_store_client.publish_measurement("THD_1W", Scalar(0.05, "%"), amp_thd_step_id)
data_store_client.publish_measurement("THD_10W", Scalar(0.12, "%"), amp_thd_step_id)

# Conditions (test parameters)
print("Publishing test conditions...")
data_store_client.publish_condition("Temperature", "double", Scalar(23.5, "°C"), ps_voltage_step_id)
data_store_client.publish_condition("Humidity", "double", Scalar(45.2, "%RH"), ps_voltage_step_id)
data_store_client.publish_condition("Input_Power", "double", Scalar(1.0, "W"), amp_gain_step_id)
data_store_client.publish_condition("Load_Impedance", "double", Scalar(8.0, "Ω"), amp_thd_step_id)

print("Sample data creation complete!")
print(f"Created {2} TestResults with {7} Steps and multiple Measurements")
print("Ready for OData queries!")

Publishing Power Supply measurements...
Publishing Amplifier measurements...
Publishing test conditions...
Sample data creation complete!
Created 2 TestResults with 7 Steps and multiple Measurements
Ready for OData queries!


## Basic Data Queries

This section demonstrates fundamental OData query operations on the data including:
- Retrieving `Measurements`
- Retrieving `Steps`

### Query All Measurements

This queries all measurements in the data store and prints their data type and associated step ID.

In [30]:
from ni.datastore import data
from ni.datastore.data import DataStoreClient

data_store_client = DataStoreClient()

# Query for all published measurements (odata_query = "")
published_measurements = data_store_client.query_measurements("")

# published_measurements is an Iterable[PublishedMeasurement]
for published_measurement in published_measurements:
    print(f"Measurement name: {published_measurement.measurement_name}")
    print(f"  Data type: {published_measurement.data_type}")
    print(f"  Step ID: {published_measurement.step_id}")

Measurement name: Output_Voltage_12V
  Data type: type.googleapis.com/ni.protobuf.types.Vector
  Step ID: 3d3241a4-bdb3-4164-a350-5ba36b7eda77
Measurement name: 50% Load Test
  Data type: type.googleapis.com/ni.protobuf.types.Vector
  Step ID: 1533463b-cd1a-4560-94f5-23bff8ffe48b
Measurement name: 1kHz THD+N Left
  Data type: type.googleapis.com/ni.protobuf.types.Vector
  Step ID: 046efda1-10fc-4df4-905e-0ade5778a798
Measurement name: 20kHz Response
  Data type: type.googleapis.com/ni.protobuf.types.Vector
  Step ID: 31dc2ef2-6e8a-434c-b207-3ccda7fc46bd
Measurement name: Max Power Left
  Data type: type.googleapis.com/ni.protobuf.types.Vector
  Step ID: 766c87cb-2257-4e9f-8565-2b359d415d04
Measurement name: Measure -12V Rail
  Data type: type.googleapis.com/ni.protobuf.types.Vector
  Step ID: 2e30090d-7a18-48cb-b36e-6fda04cf2cb3
Measurement name: Max Power Left
  Data type: type.googleapis.com/ni.protobuf.types.Vector
  Step ID: 2214e5cb-c01f-466b-b086-97d349a913f2
Measurement name: Fu

### Query All Steps

This queries all steps in the data store and prints their name and associated test result ID.

In [31]:
# Query for all steps (odata_query = "")
steps = data_store_client.query_steps("")

# data is an Iterable[Step]
for step in steps:
    print(f"Step name: {step.step_name}")
    print(f"  Test result ID: {step.test_result_id}")


Step name: Power On Sequence
  Test result ID: 1e01adf1-42c3-4f13-885a-6617fff3cfac
Step name: Output Voltage Verification
  Test result ID: 1e01adf1-42c3-4f13-885a-6617fff3cfac
Step name: Current Limit Test
  Test result ID: 1e01adf1-42c3-4f13-885a-6617fff3cfac
Step name: Output Ripple Measurement
  Test result ID: 1e01adf1-42c3-4f13-885a-6617fff3cfac
Step name: Gain Measurement
  Test result ID: c36a421d-275a-4ec4-a89c-8a126cdc1733
Step name: Frequency Response
  Test result ID: c36a421d-275a-4ec4-a89c-8a126cdc1733
Step name: THD Analysis
  Test result ID: c36a421d-275a-4ec4-a89c-8a126cdc1733
Step name: Initialize
  Test result ID: 379def37-aa19-4a90-bb5e-9a04fd0515f3
Step name: Input Voltage Test
  Test result ID: 379def37-aa19-4a90-bb5e-9a04fd0515f3
Step name: Output Voltage Test
  Test result ID: 379def37-aa19-4a90-bb5e-9a04fd0515f3
Step name: Load Regulation
  Test result ID: 379def37-aa19-4a90-bb5e-9a04fd0515f3
Step name: Ripple Measurement
  Test result ID: 379def37-aa19-4a90-b

## Test Results Hierarchy

This code demonstrates querying data to produce a hierarchy of `TestResult`, `Step`, and `Measurement`. It calls `query_steps` with an empty query to get all of the `Steps` in the data store and collects their unique test result IDs. Then for each test result ID, it calls `query_steps` filtering with `testresultid` to get all steps for that test result. Finally, it calls `query_measurements` and `query_conditions`, filtering by `stepid` to get all measurements and conditions for those `Steps`. It prints these in a hierarchy of `TestResult` -> `Steps` -> `Measurements` to illustrate how these are related.

In [41]:
print("\n" + "="*60)
print("HIERARCHICAL VIEW OF TEST DATA")
print("="*60)

# Get unique test result IDs from steps, then get the test results
all_steps = data_store_client.query_steps("")
unique_test_result_ids = set()
for step in all_steps:
    unique_test_result_ids.add(step.test_result_id)

# Get each test result by ID and organize data hierarchically
for test_result_id in unique_test_result_ids:
    test_result = data_store_client.get_test_result(test_result_id)
    print(f"\nTestResult: '{test_result.test_result_name}' (ID: {test_result.test_result_id})")
    
    # Get steps for this test result
    steps_for_result = data_store_client.query_steps(f"$filter=testresultid eq {test_result.test_result_id}")
    
    for step in steps_for_result:
        print(f"├── Step: '{step.step_name}' (ID: {step.step_id})")
        
        # Get measurements for this step
        measurements_for_step = data_store_client.query_measurements(f"$filter=stepid eq {step.step_id}")
        
        measurements_list = list(measurements_for_step)
        for i, measurement in enumerate(measurements_list):
            is_last_measurement = (i == len(measurements_list) - 1)
            prefix = "└──" if is_last_measurement else "├──"
            print(f"│   {prefix} Measurement: '{measurement.measurement_name}' ({measurement.data_type})")
        
        # Get conditions for this step
        conditions_for_step = data_store_client.query_conditions(f"$filter=stepid eq {step.step_id}")
        
        conditions_list = list(conditions_for_step)
        for i, condition in enumerate(conditions_list):
            is_last_condition = (i == len(conditions_list) - 1) and len(measurements_list) == 0
            prefix = "└──" if is_last_condition else "├──"
            print(f"│   {prefix} Condition: '{condition.condition_name}' ({condition.condition_type})")



HIERARCHICAL VIEW OF TEST DATA

TestResult: 'Audio Amplifier Performance Test' (ID: c36a421d-275a-4ec4-a89c-8a126cdc1733)
├── Step: 'Gain Measurement' (ID: 3ea54ea8-7b00-41e3-ab5d-8dc419f3317a)
│   ├── Measurement: 'Gain_1kHz' (type.googleapis.com/ni.protobuf.types.Vector)
│   └── Measurement: 'Gain_10kHz' (type.googleapis.com/ni.protobuf.types.Vector)
│   ├── Condition: 'Input_Power' (double)
├── Step: 'Frequency Response' (ID: 10444559-94d4-4e47-8ef6-998d538021e9)
│   └── Measurement: 'Frequency_Response' (type.googleapis.com/ni.protobuf.types.DoubleAnalogWaveform)
├── Step: 'THD Analysis' (ID: 721bcaec-0ad2-4419-989b-82ab0ea6714d)
│   ├── Measurement: 'THD_1W' (type.googleapis.com/ni.protobuf.types.Vector)
│   └── Measurement: 'THD_10W' (type.googleapis.com/ni.protobuf.types.Vector)
│   ├── Condition: 'Load_Impedance' (double)

TestResult: 'Audio Amplifier Performance Test' (ID: f679b40d-acd1-4600-8d55-6e2147085add)
├── Step: 'Gain Measurement' (ID: e7514c97-56a2-48ec-8b32-4dcd2261

## Basic Metadata Queries

Now that we've seen how to work with data, this section shows basic metadata queries to retrieve information on the metadata in the data store.

### Query Operators

Uses `query_operators` to retrieve the operators. It also shows how to filter operators by name.

**Additional operator query examples:**

```python
# Filter by exact operator name
operators_exact = metadata_store_client.query_operators("$filter=Name eq 'Alex Smith'")

# Filter by operator role
operators_by_role = metadata_store_client.query_operators("$filter=Role eq 'Test Engineer'")

# Filter by role containing text
operators_role_contains = metadata_store_client.query_operators("$filter=contains(Role,'Engineer')")

# Filter by name starting with text
operators_name_starts = metadata_store_client.query_operators("$filter=startswith(Name,'Alex')")

# Filter by name ending with text
operators_name_ends = metadata_store_client.query_operators("$filter=endswith(Name,'Smith')")

# Filter by specific ID (using guid format)
operators_by_id = metadata_store_client.query_operators("$filter=Id eq guid'12345678-1234-1234-1234-123456789abc'")

# Combine multiple conditions with 'and'
operators_combined = metadata_store_client.query_operators("$filter=contains(Name,'Alex') and Role eq 'Test Engineer'")
```


In [33]:
from ni.datastore.metadata import MetadataStoreClient

metadata_store_client = MetadataStoreClient()

# Query all operators
operators = metadata_store_client.query_operators("")

print("All operators:")
for operator in operators:
    print(f"  {operator.operator_name}")

print("\nFiltered operators (by name containing 'Smith'):")
filtered_operators = metadata_store_client.query_operators("$filter=contains(Name,'Smith')")
for operator in filtered_operators:
    print(f"  {operator.operator_name}")



All operators:
  Alex Smith
  Jordan Chen
  Taylor Johnson

Filtered operators (by name containing 'Smith'):
  Alex Smith


### Query Test Stations

Uses `query_test_stations` to retrieve the test stations. It also shows how to filter test stations by name.

**Additional test station query examples:**

```python
# Filter by exact test station name
stations_exact = metadata_store_client.query_test_stations("$filter=Name eq 'TestStation_A1'")

# Filter by test station name containing text
stations_name_contains = metadata_store_client.query_test_stations("$filter=contains(Name,'Station')")

# Filter by name starting with text
stations_name_starts = metadata_store_client.query_test_stations("$filter=startswith(Name,'Test')")

# Filter by name ending with text
stations_name_ends = metadata_store_client.query_test_stations("$filter=endswith(Name,'A1')")

# Filter by specific ID (using guid format)
stations_by_id = metadata_store_client.query_test_stations("$filter=Id eq guid'12345678-1234-1234-1234-123456789abc'")

# Combine multiple conditions with 'and'
stations_combined = metadata_store_client.query_test_stations("$filter=contains(Name,'Test') and contains(Name,'A1')")
```

In [34]:
# Query all test stations
test_stations = metadata_store_client.query_test_stations("")

print("All test stations:")
for station in test_stations:
    print(f"  {station.test_station_name}")

print("\nFiltered test stations (by name containing 'A1'):")
filtered_stations = metadata_store_client.query_test_stations("$filter=contains(Name,'A1')")
for station in filtered_stations:
    print(f"  {station.test_station_name}")

All test stations:
  TestStation_A1
  TestStation_B2
  TestStation_C3

Filtered test stations (by name containing 'A1'):
  TestStation_A1


### Query Hardware Items

Uses `query_hardware_items` to retrieve the hardware items (test equipment). It also shows how to filter hardware items by various properties like manufacturer, model, and serial number.

**Additional hardware item query examples:**

```python
# Filter by exact manufacturer
hardware_by_manufacturer = metadata_store_client.query_hardware_items("$filter=Manufacturer eq 'NI'")

# Filter by exact model
hardware_by_model = metadata_store_client.query_hardware_items("$filter=Model eq 'PXIe-4081'")

# Filter by serial number
hardware_by_serial = metadata_store_client.query_hardware_items("$filter=SerialNumber eq 'DMM001'")

# Filter by manufacturer containing text
hardware_manufacturer_contains = metadata_store_client.query_hardware_items("$filter=contains(Manufacturer,'NI')")

# Filter by model starting with text
hardware_model_starts = metadata_store_client.query_hardware_items("$filter=startswith(Model,'PXIe')")

# Filter by part number (if available)
hardware_by_part = metadata_store_client.query_hardware_items("$filter=PartNumber eq 'DMM-001-XYZ'")

# Combine multiple conditions - find NI PXIe instruments
hardware_combined = metadata_store_client.query_hardware_items("$filter=Manufacturer eq 'NI' and contains(Model,'PXIe')")

# Filter by calibration due date (if available)
# hardware_cal_due = metadata_store_client.query_hardware_items("$filter=CalibrationDueDate gt '2024-12-31'")
```

In [35]:
# Query all hardware items
hardware_items = metadata_store_client.query_hardware_items("")

print("All hardware items:")
for item in hardware_items:
    print(f"  {item.manufacturer} {item.model} (S/N: {item.serial_number})")

print("\nFiltered hardware items (by manufacturer 'NI'):")
filtered_hardware = metadata_store_client.query_hardware_items("$filter=Manufacturer eq 'NI'")
for item in filtered_hardware:
    print(f"  {item.manufacturer} {item.model} (S/N: {item.serial_number})")

print("\nFiltered hardware items (by model containing 'PXIe'):")
model_filtered = metadata_store_client.query_hardware_items("$filter=contains(Model,'PXIe')")
for item in model_filtered:
    print(f"  {item.manufacturer} {item.model} (S/N: {item.serial_number})")

All hardware items:
  Keysight 34461A (S/N: DMM001)
  Tektronix MSO58 (S/N: SCOPE001)
  NI PXIe-4081 (S/N: DMM001)
  NI PXIe-5171 (S/N: SCOPE001)

Filtered hardware items (by manufacturer 'NI'):
  NI PXIe-4081 (S/N: DMM001)
  NI PXIe-5171 (S/N: SCOPE001)

Filtered hardware items (by model containing 'PXIe'):
  NI PXIe-4081 (S/N: DMM001)
  NI PXIe-5171 (S/N: SCOPE001)


### Query UUTs (Units Under Test)

Uses `query_uuts` to retrieve the UUT definitions (product types being tested). It also shows how to filter UUTs by name and family.

**Additional UUT query examples:**

```python
# Filter by exact UUT name (model)
uuts_by_name = metadata_store_client.query_uuts("$filter=Name eq 'PowerSupply v2.1'")

# Filter by UUT name containing text
uuts_name_contains = metadata_store_client.query_uuts("$filter=contains(Name,'PowerSupply')")

# Filter by name starting with text
uuts_name_starts = metadata_store_client.query_uuts("$filter=startswith(Name,'Audio')")

# Filter by name ending with text
uuts_name_ends = metadata_store_client.query_uuts("$filter=endswith(Name,'v1.3')")

# Filter by specific ID (using guid format)
uuts_by_id = metadata_store_client.query_uuts("$filter=Id eq guid'12345678-1234-1234-1234-123456789abc'")

# Combine multiple conditions
uuts_combined = metadata_store_client.query_uuts("$filter=contains(Name,'Amplifier') and contains(Name,'v1')")
```

In [36]:
# Query all UUTs
uuts = metadata_store_client.query_uuts("")

print("All UUTs:")
for uut in uuts:
    print(f"  {uut.model_name}")

print("\nFiltered UUTs (by name containing 'Power'):")
filtered_uuts = metadata_store_client.query_uuts("$filter=contains(Name,'Power')")
for uut in filtered_uuts:
    print(f"  {uut.model_name}")

print("\nFiltered UUTs (by name containing 'Amplifier'):")
amplifier_uuts = metadata_store_client.query_uuts("$filter=contains(Name,'Amplifier')")
for uut in amplifier_uuts:
    print(f"  {uut.model_name}")

All UUTs:
  PowerSupply v2.1
  Audio Amplifier v1.3

Filtered UUTs (by name containing 'Power'):
  PowerSupply v2.1

Filtered UUTs (by name containing 'Amplifier'):
  Audio Amplifier v1.3


### Query UUT Instances

Uses `query_uut_instances` to retrieve specific UUT instances (individual devices with serial numbers). It also shows how to filter UUT instances by serial number and UUT ID.

**Additional UUT instance query examples:**

```python
# Filter by exact serial number
instances_by_serial = metadata_store_client.query_uut_instances("$filter=SerialNumber eq 'PS-2024-001'")

# Filter by serial number containing text
instances_serial_contains = metadata_store_client.query_uut_instances("$filter=contains(SerialNumber,'2024')")

# Filter by serial number starting with text
instances_serial_starts = metadata_store_client.query_uut_instances("$filter=startswith(SerialNumber,'PS')")

# Filter by serial number ending with text
instances_serial_ends = metadata_store_client.query_uut_instances("$filter=endswith(SerialNumber,'001')")

# Filter by manufacture date (if available)
# instances_by_date = metadata_store_client.query_uut_instances("$filter=ManufactureDate eq '2024-01-15'")

# Filter by UUT ID (reference to parent UUT)
# instances_by_uut = metadata_store_client.query_uut_instances("$filter=UutId eq guid'12345678-1234-1234-1234-123456789abc'")

# Combine multiple conditions
instances_combined = metadata_store_client.query_uut_instances("$filter=contains(SerialNumber,'AMP') and contains(SerialNumber,'2024')")
```

In [37]:
# Query all UUT instances
uut_instances = metadata_store_client.query_uut_instances("")

print("All UUT instances:")
for instance in uut_instances:
    print(f"  Serial: {instance.serial_number}")

print("\nFiltered UUT instances (by serial containing '2024'):")
filtered_instances = metadata_store_client.query_uut_instances("$filter=contains(SerialNumber,'2024')")
for instance in filtered_instances:
    print(f"  Serial: {instance.serial_number}")

print("\nFiltered UUT instances (by serial starting with 'PS'):")
ps_instances = metadata_store_client.query_uut_instances("$filter=startswith(SerialNumber,'PS')")
for instance in ps_instances:
    print(f"  Serial: {instance.serial_number}")

All UUT instances:
  Serial: PS-2024-001
  Serial: AMP-2024-042
  Serial: PS-2024-002
  Serial: AMP-2024-001

Filtered UUT instances (by serial containing '2024'):
  Serial: PS-2024-001
  Serial: AMP-2024-042
  Serial: PS-2024-002
  Serial: AMP-2024-001

Filtered UUT instances (by serial starting with 'PS'):
  Serial: PS-2024-001
  Serial: PS-2024-002


### Query Software Items

Uses `query_software_items` to retrieve software items (applications, environments, tools used in testing). It also shows how to filter software items by product and version.

**Additional software item query examples:**

```python
# Filter by exact product name
software_by_product = metadata_store_client.query_software_items("$filter=Product eq 'Python'")

# Filter by exact version
software_by_version = metadata_store_client.query_software_items("$filter=Version eq '3.11.5'")

# Filter by product containing text
software_product_contains = metadata_store_client.query_software_items("$filter=contains(Product,'Python')")

# Filter by version starting with text
software_version_starts = metadata_store_client.query_software_items("$filter=startswith(Version,'3.')")

# Filter by version ending with text
software_version_ends = metadata_store_client.query_software_items("$filter=endswith(Version,'.5')")

# Filter by specific ID (using guid format)
software_by_id = metadata_store_client.query_software_items("$filter=Id eq guid'12345678-1234-1234-1234-123456789abc'")

# Combine multiple conditions - find Python 3.x versions
software_combined = metadata_store_client.query_software_items("$filter=Product eq 'Python' and startswith(Version,'3.')")
```

In [38]:
# Query all software items
software_items = metadata_store_client.query_software_items("")

print("All software items:")
for item in software_items:
    print(f"  {item.product} {item.version}")

print("\nFiltered software items (by product 'Python'):")
filtered_software = metadata_store_client.query_software_items("$filter=Product eq 'Python'")
for item in filtered_software:
    print(f"  {item.product} {item.version}")

print("\nFiltered software items (by version starting with '3.'):")
version_filtered = metadata_store_client.query_software_items("$filter=startswith(Version,'3.')")
for item in version_filtered:
    print(f"  {item.product} {item.version}")

All software items:
  Python 3.11.5
  pytest 7.4.0
  NI-DAQmx 23.3.0

Filtered software items (by product 'Python'):
  Python 3.11.5

Filtered software items (by version starting with '3.'):
  Python 3.11.5


### Query Aliases

Uses `query_aliases` to retrieve aliases (human-readable references to entities). It also shows how to filter aliases by name and target type.

**Additional alias query examples:**

```python
# Filter by exact alias name
aliases_by_name = metadata_store_client.query_aliases("$filter=Name eq 'Operator_Smith'")

# Filter by alias name containing text
aliases_name_contains = metadata_store_client.query_aliases("$filter=contains(Name,'Smith')")

# Filter by name starting with text
aliases_name_starts = metadata_store_client.query_aliases("$filter=startswith(Name,'Operator')")

# Filter by name ending with text
aliases_name_ends = metadata_store_client.query_aliases("$filter=endswith(Name,'_A1')")

# Filter by target type (what kind of entity the alias points to)
aliases_by_target_type = metadata_store_client.query_aliases("$filter=TargetType eq DataStore.AliasTargetType'Operator'")

# Filter by target ID (specific entity the alias points to)
# aliases_by_target_id = metadata_store_client.query_aliases("$filter=TargetId eq guid'12345678-1234-1234-1234-123456789abc'")

# Combine multiple conditions - find all operator aliases
aliases_combined = metadata_store_client.query_aliases("$filter=contains(Name,'Operator') and TargetType eq DataStore.AliasTargetType'Operator'")
```

In [39]:
# Query all aliases
aliases = metadata_store_client.query_aliases("")

print("All aliases:")
for alias in aliases:
    print(f"  {alias.alias_name} -> {alias.target_type}")

print("\nFiltered aliases (by name containing 'Operator'):")
filtered_aliases = metadata_store_client.query_aliases("$filter=contains(Name,'Operator')")
for alias in filtered_aliases:
    print(f"  {alias.alias_name} -> {alias.target_type}")

print("\nFiltered aliases (by name starting with 'UUT'):")
uut_aliases = metadata_store_client.query_aliases("$filter=startswith(Name,'UUT')")
for alias in uut_aliases:
    print(f"  {alias.alias_name} -> {alias.target_type}")

All aliases:
  Operator_Smith -> 5
  TestStation_A1 -> 8
  UUT_PowerSupply_v2 -> 2
  UUT_Amplifier_v1 -> 2
  DMM_34461A -> 3
  Scope_MSO58 -> 3
  Python_3.11 -> 4
  Station_A1 -> 8
  UUT_PowerSupply_v2.1 -> 2
  DMM_PXIe4081 -> 3
  Scope_PXIe5171 -> 3

Filtered aliases (by name containing 'Operator'):
  Operator_Smith -> 5

Filtered aliases (by name starting with 'UUT'):
  UUT_PowerSupply_v2 -> 2
  UUT_Amplifier_v1 -> 2
  UUT_PowerSupply_v2.1 -> 2
